Let's start with importing the necessary modules and functions as well as the dataset.

TO DO LIST:
- generaliser filtres 3 sigma à toutes les variables
- matrice de correl
- analyse bivariee (graphes 2 à 2)

In [106]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV

import plotly.express as px


In [107]:
df = pd.read_csv("./Walmart_Store_sales.csv")

## Exploration

Now that the dataset has been imported into a Pandas dataframe, we can start exploring the data (EDA), perform some visualizations and decide how we should clean the data.

In [108]:
print("A first view of the dataset")
print(df.head())
print("Some info about the columns and their types")
print(df.info())
print("A few statistics (e.g. to identify outliers)")
print(df.describe(include="all"))
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

A first view of the dataset
   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0    6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1   13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
2   17.0  27-07-2012           NaN           0.0          NaN         NaN   
3   11.0         NaN    1244390.03           0.0        84.57         NaN   
4    6.0  28-05-2010    1644470.66           0.0        78.89       2.759   

          CPI  Unemployment  
0  214.777523         6.858  
1  128.616064         7.470  
2  130.719581         5.936  
3  214.556497         7.346  
4  212.412888         7.092  
Some info about the columns and their types
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Week

Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [109]:
df.Store.value_counts()

Store
3.0     15
1.0     11
18.0    10
19.0     9
5.0      9
14.0     9
13.0     9
7.0      8
17.0     8
2.0      8
8.0      8
6.0      7
20.0     7
4.0      7
12.0     5
10.0     5
15.0     4
16.0     4
9.0      4
11.0     3
Name: count, dtype: int64

## Analysis

An outlier is defined as a value outside of the (average +/- 3 * standard_deviation) interval for the purpose of this analysis.

From there we can already see a few issues to tackle during the pre-processing phase:
- Store column is likely to be the internal number to identify each store. It is irrelevant whether it is an int or a float because it will be a categorical variable.
- Date column has quite a few missing values (12%) and needs to be converted to date format
- Some new columns should be created to get the relevant information out of the Date column: Year, Month, Day, Weekday
- Weekly_Sales has some missing values: this is an issue because this is the target feature, therefore the corresponding records should be dropped
- Holiday_Flag gives us information about whether the day was a holiday (1) or not (0). The missing values should either be checked against an official holiday list (could be State dependant) or just set to 0 (the mode). It is a categorical variable therefore the current encoding (float, int, boolean...) is irrelevant.
- Temperature is likely to be in Fahrenheit and contains no outliers, but a few missing values (to be replaced with median price)
- Fuel_Price has some missing values (to be replaced with median price) and no outliers
- CPI is the Consumer Price Index. It has some missing values (to be replaced with median CPI) and no outliers
- Unemployment has some missing values (to be replaced with median Unemployment) and some outliers over the upper bound

## Preprocessing

We now start with the preprocessing:
- cleaning the data according to the observations above
- separating our data (according to the selected features and target)
- separating our features between categorical and numerical
- splitting the data into a train subset and a test subset
- preparing our preprocessor Pipeline and applying it -- note: the imputing will be done during this phase

In [110]:
# Dropping the records corresponding to missing values in our target column

df = df.drop(df[df.Weekly_Sales.isnull()].index)


In [111]:
df['New_date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
df['Year'] = df['New_date'].dt.year
df['Month'] = df['New_date'].dt.month
df['Day'] = df['New_date'].dt.day
df['Weekday'] = df['New_date'].dt.weekday
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,New_date,Year,Month,Day,Weekday
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011-02-18,2011.0,2.0,18.0,4.0
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011-03-25,2011.0,3.0,25.0,4.0
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaT,NaN,NaN,NaN,NaN
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010-05-28,2010.0,5.0,28.0,4.0
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010-05-28,2010.0,5.0,28.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,14.0,18-06-2010,2248645.59,0.0,72.62,2.780,182.442420,8.899,2010-06-18,2010.0,6.0,18.0,4.0
146,7.0,NaN,716388.81,NaN,20.74,2.778,NaN,NaN,NaT,NaN,NaN,NaN,NaN
147,17.0,11-06-2010,845252.21,0.0,57.14,2.841,126.111903,NaN,2010-06-11,2010.0,6.0,11.0,4.0
148,8.0,12-08-2011,856796.10,0.0,86.05,3.638,219.007525,NaN,2011-08-12,2011.0,8.0,12.0,4.0


In [112]:
# We can now drop the useless (and redundant) date columns

df = df.drop(['Date', 'New_date'], axis=1)
df

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekday
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
145,14.0,2248645.59,0.0,72.62,2.780,182.442420,8.899,2010.0,6.0,18.0,4.0
146,7.0,716388.81,NaN,20.74,2.778,NaN,NaN,NaN,NaN,NaN,NaN
147,17.0,845252.21,0.0,57.14,2.841,126.111903,NaN,2010.0,6.0,11.0,4.0
148,8.0,856796.10,0.0,86.05,3.638,219.007525,NaN,2011.0,8.0,12.0,4.0


In [113]:
# Dropping outliers

u_mean = df['Unemployment'].mean()
u_std = df['Unemployment'].std()
u_thres = u_mean + 3*u_std
df = df.drop(df[df.Unemployment > u_thres].index)

In [114]:
# Separating our features from our target

X = df.drop('Weekly_Sales', axis=1)
Y = df['Weekly_Sales']

In [115]:
# Separating numerical from categorical data
# This will be performed by hand as we cannot rely on the datatypes from the dataframe

numerical_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Weekday']
categorical_features = ['Store', 'Holiday_Flag']

In [116]:
# Splitting the data between train set and test set

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1337)

In [117]:
# Pipelines for our preprocessing:
# - Imputing strategy is median for numerical and mode for categorical
# - StandardScaler for numerical features, OneHotEncoder to create categories for categorical features

numerical_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="median"),
        ),  # missing values will be replaced by columns' median
        ("scaler",
         StandardScaler()), # standardization of numerical data
    ]
)

categorical_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="most_frequent"),
        ),  # missing values will be replaced by most frequent value
        (
            "encoder",
            OneHotEncoder(drop="first"),
        ),  # first column will be dropped to avoid creating correlations between features
    ]
)

In [118]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

## Regression

In [119]:
# Applying the preprocessing to our features

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [120]:
# We perform a standard linear regression on our data

regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression()

In [121]:
# Print R2 scores (let's save this information for later)

lreg_score_train = regressor.score(X_train, Y_train)
lreg_score_test = regressor.score(X_test, Y_test)

print("R2 score on training set : ", lreg_score_train)
print("R2 score on test set : ", lreg_score_test)

R2 score on training set :  0.975096387576618
R2 score on test set :  0.9358596157622834


## How can we assess the quality of our regressor ? Are we overfitting ?

At first glance the R2 scores are excellent but the R2 score on the training set is quite high so there's always a suspicion of overfitting. We are going to look at the cross valuation score and in particular the standard deviation.

In [122]:
scores = cross_val_score(regressor, X_train, Y_train, cv = 10)

print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

The cross-validated R2-score is :  0.9372049909510551
The standard deviation is :  0.030167175142772878


The standard deviation on our cross validation is quite low. In particular we notice that (R2 score on test + std dev) is higher than (R2 score on train - std dev). Since there is no overlap we can assume that there is actually no overfitting.

This being said, we can always try to optimize our model by using regularization techniques (Lasso and Ridge).

## Regularization

We have 2 possibile choices at this step: either we start with Ridge or with Lasso. Given their characteristics, it is usual to start with Lasso as it can help us perform some feature selection. As we force a stricter and stricter regularization, Lasso will use fewer and fewer features. This can help us detect which features are absolutely essential to our model, and feed them to Ridge to obtain a final model (supposedly and hopefully of higher quality than the original model).

Befoire going into feature selection with Lasso, we can first have a look at the coefficients from our previous linear regression :

In [123]:
column_names = []

for (
    name,
    step,
    features_list,
) in preprocessor.transformers_:  # loop over steps of ColumnTransformer
    if name == "num":  # if pipeline is for numeric variables
        features = (
            features_list  # just get the names of columns to which it has been applied
        )
    else:  # if pipeline is for categorical variables
        features = (
            step.get_feature_names_out()
        )  # get output columns names from OneHotEncoder
    column_names.extend(features)  # concatenate features names

print("Names of columns corresponding to each coefficient: ", column_names)

# Create a pandas DataFrame with the coefficients from the regressor

coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])

# Visualization of the relative importance of the coefficients

# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Weekday', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0', 'Holiday_Flag_1.0']


Sadly this doesn't seem to be telling us much. The stores seem to be good predictors of weekly sales: we can assume that a specific store will have rather stable weekly sales. Other features seem much less relevant, in particular which day of the week it is or whether it's a holiday. As you target is weekly sales and not daily sales, this seems understandable.

Let's start with a basic Lasso regularization with a parameter $\alpha = 1$.

In [124]:
first_lasso = Lasso(alpha = 1)
first_lasso.fit(X_train, Y_train)

# Print R^2 scores
print("R2 score on training set : ", first_lasso.score(X_train, Y_train), "vs unreg linear regression score on training set :", lreg_score_train)
print("R2 score on test set : ", first_lasso.score(X_test, Y_test), "vs unreg linear regression score on test set :", lreg_score_test)

R2 score on training set :  0.9750963856208708 vs unreg linear regression score on training set : 0.975096387576618
R2 score on test set :  0.9358602293771652 vs unreg linear regression score on test set : 0.9358596157622834


As we can see the scores are extremely close. Let'us GridSearchCV to optimize our hyperparameter (alpha).

In [125]:
# Perform grid search
print("Grid search...")
lasso_cv = Lasso(max_iter=10000)

# Grid of values to be tested
params = {
    'alpha': [1, 2, 5, 10, 50, 100, 200, 300, 400, 500, 1000]
}

best_lasso = GridSearchCV(lasso_cv, param_grid = params, cv = 3) # cv : the number of folds to be used for CV

best_lasso.fit(X_train, Y_train) # training the best lasso on the whole training data

print("...Done.")
print("Best hyperparameters : ", best_lasso.best_params_)
print("R2 score on training set : ", best_lasso.score(X_train, Y_train), "vs unreg linear regression score on training set :", lreg_score_train)
print("R2 score on test set : ", best_lasso.score(X_test, Y_test), "vs unreg linear regression score on test set :", lreg_score_test)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 300}
R2 score on training set :  0.9749339820091621 vs unreg linear regression score on training set : 0.975096387576618
R2 score on test set :  0.9353737211882552 vs unreg linear regression score on test set : 0.9358596157622834


Again the scores are very close to the ones we achieved with our unregularized linear regression.

In [126]:
for i, coef in enumerate(best_lasso.best_estimator_.coef_):
    if coef == 0:
        print("Coef 0 for feature :", column_names[i])


Coef 0 for feature : Weekday
Coef 0 for feature : Holiday_Flag_1.0


This confirms what we had noticed before, but Lasso couldn't help us further in performing feature selection. All the remaining features are key to obtaining a good model.

Let's check if a Ridge regularization could do better without any feature selection. We will directly perform a GridSearchCV on the Ridge regressor.

In [127]:
# Perform grid search
print("Grid search...")
ridge_cv = Ridge()

# Grid of values to be tested
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
}

first_ridge = GridSearchCV(ridge_cv, param_grid = params, cv = 3) # cv : the number of folds to be used for CV

first_ridge.fit(X_train, Y_train) # training the best lasso on the whole training data

print("...Done.")
print("Best hyperparameters : ", first_ridge.best_params_)
print("R2 score on training set : ", first_ridge.score(X_train, Y_train), "vs unreg linear regression score on training set :", lreg_score_train)
print("R2 score on test set : ", first_ridge.score(X_test, Y_test), "vs unreg linear regression score on test set :", lreg_score_test)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.01}
R2 score on training set :  0.9750831158037975 vs unreg linear regression score on training set : 0.975096387576618
R2 score on test set :  0.9359100313588087 vs unreg linear regression score on test set : 0.9358596157622834


The Ridge regularization performs only barely better than the previous models.

We can now drop these 2 features and check if Ridge gives us a better model. Again we will directly perform a GridSearchCV on the Ridge regressor.

In [128]:
# Separating our features from our target

X = df.drop(['Weekly_Sales', 'Weekday', 'Holiday_Flag'], axis=1)
Y = df['Weekly_Sales']

# Separating numerical from categorical data
# This will be performed by hand as we cannot rely on the datatypes from the dataframe

numerical_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day']
categorical_features = ['Store']

# Splitting the data between train set and test set

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1337)

# Pipelines for our preprocessing:
# - Imputing strategy is median for numerical and mode for categorical
# - StandardScaler for numerical features, OneHotEncoder to create categories for categorical features

numerical_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="median"),
        ),  # missing values will be replaced by columns' median
        ("scaler",
         StandardScaler()), # standardization of numerical data
    ]
)

categorical_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="most_frequent"),
        ),  # missing values will be replaced by most frequent value
        (
            "encoder",
            OneHotEncoder(drop="first"),
        ),  # first column will be dropped to avoid creating correlations between features
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Applying the preprocessing to our features

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Perform grid search
print("Grid search...")
ridge_cv = Ridge()

# Grid of values to be tested
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
}

best_ridge = GridSearchCV(ridge_cv, param_grid = params, cv = 3) # cv : the number of folds to be used for CV

best_ridge.fit(X_train, Y_train) # training the best lasso on the whole training data

print("...Done.")
print("Best hyperparameters : ", best_ridge.best_params_)
print("R2 score on training set : ", best_ridge.score(X_train, Y_train), "vs unreg linear regression score on training set :", lreg_score_train)
print("R2 score on test set : ", best_ridge.score(X_test, Y_test), "vs unreg linear regression score on test set :", lreg_score_test)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.01}
R2 score on training set :  0.9750759605253774 vs unreg linear regression score on training set : 0.975096387576618
R2 score on test set :  0.9361224372034673 vs unreg linear regression score on test set : 0.9358596157622834


## Conclusion

Once we had cleaned and prepared our data, our first, unregularized linear regression performed very well in predicting the target value ("Weekly Sales") based on the features provided. The cross validation score and standard deviation as well as a study of the regression coefficients seemed to indicate little to no overfitting of this model.

A feature selection approach based on coefficients was confirmed by a Lasso regularization. Once the two identified columns were dropped, a Ridge regularization (with a properly optimized hyperparameter) performed slightly (but barely) better than the initial linear regression or than a Ridge regularization based on all available features.